First I need to get my data 

In [1002]:
import os
import pandas as pd
import numpy as np
import mypytable as mypy
from sklearn.model_selection import train_test_split
import tensorflow as tf
import copy


In [1003]:
stockfiles= os.listdir("/home/cbarker4/Documents/DataScience/StockTrader/Data")
# print(stockfiles)

In [1004]:
X = []
Y=[]
Xlast=[]
stockfiles=["AAPL.csv"]
for j,val in enumerate(stockfiles):
    print(val)
    if j == 30:
        break

    mt = mypy.MyPyTable()
    mt.load_from_file("/home/cbarker4/Documents/DataScience/StockTrader/Data/"+val)
    # mt.drop_column('v')
    # mt.drop_column('t')
    # mt.drop_column('s')
    # mt.drop_column('h')
    # mt.drop_column('l')
    # mt.drop_column('o')

    mt.drop_column(0)
    i = 100
    while i <  len(mt.data)-1:
    

        table = mt.create_sub_table(i-100,i)
        # big = table.max()
        # little = table.min()
        # table = table.normalize()
        

      
        # print(table.data[0])
        X.append(copy.deepcopy(table.get_column(0)))
        # Xlast.append(table.data[-1])
        # num = int((((mt.get_row(i+1)[0])-little))/(big-little)*1000)/10
        num = mt.get_row(i+1)[0]
        Y.append(num)
        i+=1
    


AAPL.csv


In [1005]:
X = np.array(X)
# print(Y)
Y = np.array(Y)
# print(Y.shape)
count = 0
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.25 ,random_state=10)
i=0
for val in X_train:
    # print(val[-1][0])
    # if val[-1][0]<Y_train[i]:
    if val[-1]<Y_train[i]:
        count+=1
    i+=1

print(X_train[-1][-1])
print(Y_train[-1])
print(count/len(Y_train))



152.37
154.48
0.4864864864864865


In [1006]:
# Check shapes
print('X_train shape: ', X_train.shape)
print('y_train shape: ', Y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', Y_test.shape)

X_train shape:  (111, 100)
y_train shape:  (111,)
X_test shape:  (38, 100)
y_test shape:  (38,)


In [1007]:
def accuracy(y_true,y_pred):

    good = abs(y_true-y_pred)
    if good>.25:
        False
    else:
        True
    


# Build epoch checkpoint callback
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=500,
        verbose=1,
        min_delta = 0,
        mode='auto',
        baseline=None,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join("/home/cbarker4/Documents/DataScience/StockTrader/Model", 'ckpt', "{epoch:02d}-{loss:.2f}.hdf5"),
        monitor='loss',
        verbose = 0,
        save_best_only = False,
        save_weights_only = False,
        mode = 'auto',
        save_freq='epoch',
        options=None,
        initial_value_threshold=None
    )   
]

In [1008]:
class Custom_Ce_Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call (self, y_true,y_pred):
        # if y_true[1]:
        #     if y_pred>y_true[0]:
        ans = tf.math.subtract(y_true[0],y_pred)
        ans = tf.math.square(ans)
        return tf.math.abs(ans)
        #     else:
        #         ans = tf.math.subtract(y_true[0],y_pred)
        #         ans =  tf.math.abs(ans)
        #         return tf.math.square(ans)
        # else:
        #     if y_pred<y_true[0]:
        #         ans = tf.math.subtract(y_true[0],y_pred)
        #         return tf.math.abs(ans)
        #     else:
        #         ans = tf.math.subtract(y_true[0],y_pred)
        #         ans =  tf.math.abs(ans)
        #         return tf.math.square(ans)

        

In [1014]:
# loss_fun=["mean_absolute_error","KLD","MAE","MAPE","MSE","MSLE","binary_crossentropy","binary_focal_crossentropy","binary_crossentropy","binary_focal_crossentropy","categorical_accuracy","categorical crossentropy","deserialize","get","logcosh","hinge","kld","poisson","mse","mean_squared_logarithmic_error"]
# loss_fun = ["binary_crossentropy","binary_focal_crossentropy"]
# loss_fun = ["binary_crossentropy","binary_focal_crossentropy","categorical_accuracy","categorical crossentropy","deserialize"]
# loss_fun = ["mean_squared_logarithmic_error"]
loss_fun = ["MAE"]
best = {}
for losfun in loss_fun:
    # break

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(units=512,
                                return_sequences=True,
                                input_shape=(X_train.shape[1],1)))
    model.add(tf.keras.layers.LSTM(units=256,return_sequences=True))
    model.add(tf.keras.layers.LSTM(64))
    model.add(tf.keras.layers.Dense(32))
    # model.add(tf.keras.layers.Dense(64))
    # model.add(tf.keras.layers.Dense(32))

    model.add(tf.keras.layers.Dropout(0.5))
    
    model.add(tf.keras.layers.Dense(1))
    model.summary()


    

    # print(Y_train)
    
    

Model: "sequential_110"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_239 (LSTM)             (None, 100, 512)          1052672   
                                                                 
 lstm_240 (LSTM)             (None, 100, 256)          787456    
                                                                 
 lstm_241 (LSTM)             (None, 64)                82176     
                                                                 
 dense_292 (Dense)           (None, 32)                2080      
                                                                 
 dropout_98 (Dropout)        (None, 32)                0         
                                                                 
 dense_293 (Dense)           (None, 1)                 33        
                                                                 
Total params: 1,924,417
Trainable params: 1,924,417


In [1010]:
model.compile(optimizer='adam',
              loss='mean_squared_error')
model.fit(X_train,Y_train,epochs=1000, callbacks=callbacks)


Epoch 1/1000
4/4 [==============================] - 2s 19ms/step - loss: 21506.5625
Epoch 2/1000
4/4 [==============================] - 0s 18ms/step - loss: 20956.6934
Epoch 3/1000
4/4 [==============================] - 0s 11ms/step - loss: 20256.3574
Epoch 4/1000
4/4 [==============================] - 0s 13ms/step - loss: 19583.2305
Epoch 5/1000
4/4 [==============================] - 0s 12ms/step - loss: 18685.0547
Epoch 6/1000
4/4 [==============================] - 0s 13ms/step - loss: 18074.9004
Epoch 7/1000
4/4 [==============================] - 0s 13ms/step - loss: 17309.1309
Epoch 8/1000
4/4 [==============================] - 0s 14ms/step - loss: 16444.1387
Epoch 9/1000
4/4 [==============================] - 0s 13ms/step - loss: 15454.7139
Epoch 10/1000
4/4 [==============================] - 0s 14ms/step - loss: 14452.4639
Epoch 11/1000
4/4 [==============================] - 0s 12ms/step - loss: 13299.5156
Epoch 12/1000
4/4 [==============================] - 0s 12ms/step - loss: 

In [1011]:
y_pred = model.predict(X_test)
print(y_pred)

2/2 [==============================] - 0s 6ms/step
[[130.52766]
 [149.90842]
 [149.62938]
 [130.72057]
 [148.39822]
 [147.80934]
 [149.10136]
 [131.50197]
 [145.57991]
 [155.24854]
 [154.95525]
 [153.48463]
 [143.48763]
 [152.56895]
 [144.24446]
 [151.22197]
 [131.14024]
 [148.80702]
 [140.61461]
 [145.39839]
 [143.47173]
 [147.71294]
 [147.90779]
 [145.16652]
 [141.14742]
 [151.87817]
 [130.00603]
 [153.85591]
 [141.68097]
 [129.8614 ]
 [150.191  ]
 [144.31963]
 [143.26201]
 [149.55551]
 [144.85551]
 [138.91316]
 [141.63098]
 [146.05745]]


In [1012]:

Xlast=[]
correct = 0
for val in X_test:
    Xlast.append(val[-1])


correct = 0
removed = 0
falsepos = 0
pos = 0
neg=0
falseneg =0
for i,val in enumerate(Y_test):
    if i < 10:
        print(Xlast[i],Y_test[i],y_pred[i])
    if Xlast[i]!=y_pred[i]:
        if Xlast[i] > val:

            if Xlast[i]>y_pred[i]:
                pos+=1
                correct+=1
            else :
                neg+=1
                falseneg+=1
        
        else:
            if Xlast[i]<y_pred[i]:
                correct+=1 
                neg+=1
            else:
                pos+=1
                falsepos+=1
    else:
        removed+=1

print('Correct predictions: ', correct)
print('Incorrect predictions: ', (len(y_pred)-removed) - correct)
print('Accuracy: ', correct / (len(y_pred)-removed))
print('False Pos:',falsepos)
print('Pos:',pos)
print('Pos Accuracy',(pos-falsepos)/pos)
print('Neg:',neg)
print('Neg Accuracy',(neg-falseneg)/neg)

126.04 129.93 [130.52766]
152.59 155.85 [149.90842]
150.7 156.9 [149.62938]
129.62 130.73 [130.72057]
150.04 150.72 [148.39822]
148.01 151.07 [147.80934]
151.07 144.22 [149.10136]
129.93 126.36 [131.50197]
144.8 153.34 [145.57991]
165.63 164.66 [155.24854]
Correct predictions:  16
Incorrect predictions:  22
Accuracy:  0.42105263157894735
False Pos: 15
Pos: 24
Pos Accuracy 0.375
Neg: 14
Neg Accuracy 0.5
